#Data Transition Device Side

##Here we implement dtata transition between Google Coral and Cloud computational server using cloud database.

### To perfrom tests, we will use test image of car plate.

In [ ]:
from PIL import Image
import numpy
 
 
img= Image.open("/content/5eb132d54307773bff169aed6d1a7df6-400x225.jpg")
np_img = numpy.array(img)

##This code runs on Google Coral, when data needs to be transmitted.

In [ ]:
import psycopg2
import pickle
import numpy as np
import time
start=time.time()
def insert_array_and_poll_results(array, id):
    # Open a connection
    conn = psycopg2.connect("host=109.120.191.148 dbname=PostgreSQL-4323 user=user password=0HD9E7L5A0410WYD")
    cursor = conn.cursor()

    # Pickle the array and insert it into the arrays table.
    pickled_array = pickle.dumps(array)
    cursor.execute("""
        INSERT INTO arrays (id, image) VALUES (%s, %s)
    """, (id, psycopg2.Binary(pickled_array)))
    conn.commit()

    while True:
        # Check the results table for a result with the same id.
        cursor.execute("""
            SELECT result FROM user_results WHERE id = %s
        """, (id,))
        row = cursor.fetchone()

        if row is not None:
            result = row[0]

            # Stop polling once we've found a result.
            break

        # Wait for a bit before checking again.
        time.sleep(0.001)

    cursor.close()
    conn.close()
    return result



# Insert the array and poll for a result.
a = insert_array_and_poll_results(np_img, 666)
end=time.time()
print(a,end-start)

1 3.132169246673584
